In [ ]:
# بسم الله الرحمن الرحیم

In [ ]:
# BaseLine for HDvoicer

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**MRH: You may need to downconvert and install some libraries to be compatible with the code**

In [ ]:
pip install "librosa==0.9.1"

In [ ]:
!pip install tensorboardX

**MRH: The Following part is used one time during feature Extraction, So, after one round you can skip this part.
Also, the addresses shouls be modified. since my address correrspond to my google drive.**

In [ ]:
# preprocess/tacotron/hyperparams.py

# -*- coding: utf-8 -*-
#/usr/bin/python2
'''
By kyubyong park. kbpark.linguist@gmail.com.
https://www.github.com/kyubyong/tacotron
'''
class Hyperparams:
    '''Hyper parameters'''

    top_db = 15

    # signal processing
    sr = 22050 # Sample rate.
    # n_fft = 2048 # fft points (samples)
    n_fft = 1024 # fft points (samples)
    # frame_shift = 0.0125 # seconds
    # frame_length = 0.05 # seconds
    # hop_length = int(sr*frame_shift) # samples.
    hop_length = 256 # samples.
    # win_length = int(sr*frame_length) # samples.
    win_length = 1024 # samples.
    n_mels = 80 # Number of Mel banks to generate
    power = 1.2 # Exponent for amplifying the predicted magnitude
    n_iter = 100 # Number of inversion iterations
    preemphasis = .97 # or None
    max_db = 100
    ref_db = 20

In [ ]:
# to test
hp = Hyperparams()
print(hp.sr)
print(hp.n_mels)

22050
80


In [ ]:
# adaptive_voice_conversion/preprocess/make_datasets_vctk.py

#

import pickle
import librosa
import sys
import glob
import random
import os
from collections import defaultdict
import re
import numpy as np
import json
# from tacotron.utils import get_spectrograms

def read_speaker_info(speaker_info_path):
    speaker_ids = []
    with open(speaker_info_path, 'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            speaker_id = line.strip().split()[0]
            speaker_ids.append(speaker_id)
    return speaker_ids


##### MRH: This is Very Important... using *.flac or *.wav

def read_filenames(root_dir):
    speaker2filenames = defaultdict(lambda : [])
    for path in sorted(glob.glob(os.path.join(root_dir, '*/p*_mic2.flac'))):
        filename = os.path.basename(path)
        match = re.match(r'p(\d+)_(\d+)_mic2\.flac', filename)

        if match:
            # THIS IS THE CRITICAL CHANGE: Prepend 'p' to the captured digits
            speaker_id = 'p' + match.groups()[0]  # match.groups()[0] gets '225', '226', etc.
            # utt_id = match.groups()[1] # You don't use this, but it would be the second group

            speaker2filenames[speaker_id].append(path)
        else:
            print(f"Warning: Filename '{filename}' matched glob but not regex. Skipping. Path: {path}")

    return speaker2filenames

# def read_filenames(root_dir):
#     speaker2filenames = defaultdict(lambda : [])
#     for path in sorted(glob.glob(os.path.join(root_dir, '*/*'))):
#         filename = path.strip().split('/')[-1]
#         speaker_id, utt_id = re.match(r'p(\d+)_(\d+)\.wav', filename).groups()
#         speaker2filenames[speaker_id].append(path)
#     return speaker2filenames


def wave_feature_extraction(wav_file, sr):
    y, sr = librosa.load(wav_file, sr)
    y, _ = librosa.effects.trim(y, top_db=20)
    return y

def spec_feature_extraction(wav_file):
    mel, mag = get_spectrograms(wav_file)
    return mel, mag

if __name__ == '__main__':
    # MRH:
    """
    data_dir = sys.argv[1]
    speaker_info_path = sys.argv[2]
    output_dir = sys.argv[3]
    test_speakers = int(sys.argv[4])
    test_proportion = float(sys.argv[5])
    sample_rate = int(sys.argv[6])
    n_utts_attr = int(sys.argv[7])
    """
    #
    output_dir="/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/"  # MRH: You may modify the address
    data_dir = "/content/drive/MyDrive/Dataset/VCTK-Corpus-MRH-Experimental/wav48/" # MRH: You may modify the address
    speaker_info_path = "/content/drive/MyDrive/Dataset/VCTK-Corpus-MRH-Experimental/speaker-info.txt" # MRH: You may modify the address
    test_speakers = 20
    test_proportion = 0.1
    n_utt_attr = 5000

    #
    os.makedirs(output_dir, exist_ok=True)
    #


    speaker_ids = read_speaker_info(speaker_info_path)
    random.shuffle(speaker_ids)

    train_speaker_ids = speaker_ids[:-test_speakers]
    test_speaker_ids = speaker_ids[-test_speakers:]

    speaker2filenames = read_filenames(data_dir)

    train_path_list, in_test_path_list, out_test_path_list = [], [], []

    for speaker in train_speaker_ids:
        # print('mrh speaker= ',speaker)
        path_list = speaker2filenames[speaker]
        # print('mrh path_list= ',path_list)
        random.shuffle(path_list)
        test_data_size = int(len(path_list) * test_proportion)
        train_path_list += path_list[:-test_data_size]
        in_test_path_list += path_list[-test_data_size:]

    with open(os.path.join(output_dir, 'in_test_files.txt'), 'w') as f:
        for path in in_test_path_list:
            f.write(f'{path}\n')

    for speaker in test_speaker_ids:
        path_list = speaker2filenames[speaker]
        out_test_path_list += path_list

    with open(os.path.join(output_dir, 'out_test_files.txt'), 'w') as f:
        for path in out_test_path_list:
            f.write(f'{path}\n')

    for dset, path_list in zip(['train', 'in_test', 'out_test'], \
            [train_path_list, in_test_path_list, out_test_path_list]):
        print(f'processing {dset} set, {len(path_list)} files')
        data = {}
        output_path = os.path.join(output_dir, f'{dset}.pkl')
        all_train_data = []
        for i, path in enumerate(sorted(path_list)):
            if i % 500 == 0 or i == len(path_list) - 1:
                print(f'processing {i} files')
            filename = path.strip().split('/')[-1]
            mel, mag = spec_feature_extraction(path)
            data[filename] = mel
            if dset == 'train' and i < n_utts_attr:
                all_train_data.append(mel)



        #
                # In your Colab Cell 1, where you define parameters:
        print(f"n_utt_attr: {n_utt_attr}") # Make sure this is a sensible number, like 1000 or more

        # In your make_datasets_vctk.py script, just before the problematic lines:

        # After speaker_ids = read_speaker_info(speaker_info_path)
        print(f"Total speakers found: {len(speaker_ids)}")
        print(f"First few speaker IDs: {speaker_ids[:5]}")

        # After train_speaker_ids = speaker_ids[:-test_speakers]
        print(f"Number of train speakers: {len(train_speaker_ids)}")
        print(f"Number of test speakers: {len(test_speaker_ids)}")

        # After speaker2filenames = read_filenames(data_dir)
        print(f"Total files found by read_filenames: {sum(len(v) for v in speaker2filenames.values())}")
        # Print a sample of files for a speaker (replace 'pXXX' with a speaker ID you expect to find)
        # if 'p225' in speaker2filenames:
        #    print(f"Sample files for p225: {speaker2filenames['p225'][:5]}")

        # Just before the loop that populates train_path_list:
        print(f"Initial train_path_list length (before populating): {len(train_path_list)}")
        print(f"Initial in_test_path_list length (before populating): {len(in_test_path_list)}")

        # After the loops that populate train_path_list and in_test_path_list:
        print(f"Final train_path_list length: {len(train_path_list)}")
        print(f"Final in_test_path_list length: {len(in_test_path_list)}")
        print(f"Final out_test_path_list length: {len(out_test_path_list)}")
        #

        # raise Exception('Mohammad Reza')





        if dset == 'train':
            all_train_data = np.concatenate(all_train_data)
            mean = np.mean(all_train_data, axis=0)
            std = np.std(all_train_data, axis=0)
            attr = {'mean': mean, 'std': std}
            with open(os.path.join(output_dir, 'attr.pkl'), 'wb') as f:
                pickle.dump(attr, f)
        for key, val in data.items():
            val = (val - mean) / std
            data[key] = val
        with open(output_path, 'wb') as f:
            pickle.dump(data, f)

In [ ]:
# reduce_dataset.py
import pickle
import sys

pkl_path = '/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/train.pkl' # MRH: You may modify the address
segment_size=128
output_path = f'/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/train_{segment_size}.pkl' # MRH: You may modify the address


if __name__ == '__main__':
    # pkl_path = sys.argv[1]
    # output_path = sys.argv[2]
    # segment_size = int(sys.argv[3])

    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)

    reduced_data = {key:val for key, val in data.items() if val.shape[0] > segment_size}

    with open(output_path, 'wb') as f:
        pickle.dump(reduced_data, f)

In [ ]:
# MRH= This is for train.pkl to produce train_samples_{segment_size}.json
import os

# Configuration variables
training_samples = 10000000
segment_size = 128
pickle_path = '/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/train.pkl'  # MRH: You may modify the address

# Construct sample_path using f-string
sample_path = f'/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/train_samples_{segment_size}.json'  # MRH: You may modify the address


print(f"Configuration set:")
print(f"  Training Samples: {training_samples}")
print(f"  Segment Size: {segment_size}")
print(f"  Pickle Path: {pickle_path}")
print(f"  Sample Path: {sample_path}")

#---------------------#

import json
import pickle
import random

# The variables (training_samples, segment_size, pickle_path, sample_path)
# are assumed to be defined in the previous cell.

# Load the data from the pickle file
print(f"Loading data from {pickle_path}...")
with open(pickle_path, 'rb') as f:
    data = pickle.load(f)
print("Data loaded successfully.")

# (utt_id, timestep)
samples = []

# Filter utterances with length > segment_size
utt_list = [key for key in data]
utt_list = sorted(list(filter(lambda u : len(data[u]) > segment_size, utt_list)))
print(f'{len(utt_list)} utterances meet the segment size criteria.')

# Generate samples
print(f"Generating {training_samples} samples...")
sample_utt_index_list = random.choices(range(len(utt_list)), k=training_samples)

for i, utt_ind in enumerate(sample_utt_index_list):
    if i % 500000 == 0: # Increased frequency for print to avoid too many outputs for large n_samples
        print(f'Sampled {i}/{training_samples} samples...')
    utt_id = utt_list[utt_ind]
    # Ensure t is within valid bounds
    t_max = len(data[utt_id]) - segment_size
    if t_max < 0: # Should not happen if filtered correctly, but good for robustness
        continue
    t = random.randint(0, t_max)
    samples.append((utt_id, t))

# Save the samples to a JSON file
print(f"Saving {len(samples)} samples to {sample_path}...")
with open(sample_path, 'w') as f:
    json.dump(samples, f)
print("Sampling complete and saved.")

In [ ]:
# MRH= This is for in_test.pkl to produce in_test_samples_{segment_size}.json
import os

# Configuration variables
testing_samples = 10000
segment_size = 128
pickle_path = '/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/in_test.pkl' # MRH: You may modify the address

# Construct sample_path using f-string
sample_path = f'/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/in_test_samples_{segment_size}.json' # MRH: You may modify the address


print(f"Configuration set:")
print(f"  Training Samples: {training_samples}")
print(f"  Segment Size: {segment_size}")
print(f"  Pickle Path: {pickle_path}")
print(f"  Sample Path: {sample_path}")

#---------------------#

import json
import pickle
import random

# The variables (testing_samples, segment_size, pickle_path, sample_path)
# are assumed to be defined in the previous cell.

# Load the data from the pickle file
print(f"Loading data from {pickle_path}...")
with open(pickle_path, 'rb') as f:
    data = pickle.load(f)
print("Data loaded successfully.")

# (utt_id, timestep)
samples = []

# Filter utterances with length > segment_size
utt_list = [key for key in data]
utt_list = sorted(list(filter(lambda u : len(data[u]) > segment_size, utt_list)))
print(f'{len(utt_list)} utterances meet the segment size criteria.')

# Generate samples
print(f"Generating {testing_samples} samples...")
sample_utt_index_list = random.choices(range(len(utt_list)), k=testing_samples)

for i, utt_ind in enumerate(sample_utt_index_list):
    if i % 500000 == 0: # Increased frequency for print to avoid too many outputs for large n_samples
        print(f'Sampled {i}/{testing_samples} samples...')
    utt_id = utt_list[utt_ind]
    # Ensure t is within valid bounds
    t_max = len(data[utt_id]) - segment_size
    if t_max < 0: # Should not happen if filtered correctly, but good for robustness
        continue
    t = random.randint(0, t_max)
    samples.append((utt_id, t))

# Save the samples to a JSON file
print(f"Saving {len(samples)} samples to {sample_path}...")
with open(sample_path, 'w') as f:
    json.dump(samples, f)
print("Sampling complete and saved.")


**MRH: As mentioned the Above part are implemented once to extract features, then the following part should be runned every time to produce resultst**

In [4]:
# adaptive_voice_conversion /utils.py
import torch
import numpy as np
from tensorboardX import SummaryWriter
import editdistance
import torch.nn as nn
import torch.nn.init as init

def cc(net):
    # print('100')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # print('200')
    return net.to(device)

class Logger(object):
    def __init__(self, logdir='./log'):
        self.writer = SummaryWriter(logdir)

    def scalar_summary(self, tag, value, step):
        self.writer.add_scalar(tag, value, step)

    def scalars_summary(self, tag, dictionary, step):
        self.writer.add_scalars(tag, dictionary, step)

    def text_summary(self, tag, value, step):
        self.writer.add_text(tag, value, step)

    def audio_summary(self, tag, value, step, sr):
        writer.add_audio(tag, value, step, sample_rate=sr)

def infinite_iter(iterable):
    it = iter(iterable)
    while True:
        try:
            ret = next(it)
            yield ret
        except StopIteration:
            it = iter(iterable)

In [ ]:
##### Model

In [5]:
# adaptive_voice_conversion/model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as ag
import numpy as np
from math import ceil
from functools import reduce
from torch.nn.utils import spectral_norm
# from utils import cc

class DummyEncoder(object):
    def __init__(self, encoder):
        self.encoder = encoder

    def load(self, target_network):
        self.encoder.load_state_dict(target_network.state_dict())

    def __call__(self, x):
        return self.encoder(x)

def pad_layer(inp, layer, pad_type='reflect'):
    kernel_size = layer.kernel_size[0]
    if kernel_size % 2 == 0:
        pad = (kernel_size//2, kernel_size//2 - 1)
    else:
        pad = (kernel_size//2, kernel_size//2)
    # padding
    inp = F.pad(inp,
            pad=pad,
            mode=pad_type)
    out = layer(inp)
    return out

def pad_layer_2d(inp, layer, pad_type='reflect'):
    kernel_size = layer.kernel_size
    if kernel_size[0] % 2 == 0:
        pad_lr = [kernel_size[0]//2, kernel_size[0]//2 - 1]
    else:
        pad_lr = [kernel_size[0]//2, kernel_size[0]//2]
    if kernel_size[1] % 2 == 0:
        pad_ud = [kernel_size[1]//2, kernel_size[1]//2 - 1]
    else:
        pad_ud = [kernel_size[1]//2, kernel_size[1]//2]
    pad = tuple(pad_lr + pad_ud)
    # padding
    inp = F.pad(inp,
            pad=pad,
            mode=pad_type)
    out = layer(inp)
    return out

def pixel_shuffle_1d(inp, scale_factor=2):
    batch_size, channels, in_width = inp.size()
    channels //= scale_factor
    out_width = in_width * scale_factor
    inp_view = inp.contiguous().view(batch_size, channels, scale_factor, in_width)
    shuffle_out = inp_view.permute(0, 1, 3, 2).contiguous()
    shuffle_out = shuffle_out.view(batch_size, channels, out_width)
    return shuffle_out

def upsample(x, scale_factor=2):
    x_up = F.interpolate(x, scale_factor=scale_factor, mode='nearest')
    return x_up

def flatten(x):
    out = x.contiguous().view(x.size(0), x.size(1) * x.size(2))
    return out

def concat_cond(x, cond):
    # x = [batch_size, x_channels, length]
    # cond = [batch_size, c_channels]
    cond = cond.unsqueeze(dim=2)
    cond = cond.expand(*cond.size()[:-1], x.size(-1))
    out = torch.cat([x, cond], dim=1)
    return out

def append_cond(x, cond):
    # x = [batch_size, x_channels, length]
    # cond = [batch_size, x_channels * 2]
    p = cond.size(1) // 2
    mean, std = cond[:, :p], cond[:, p:]
    out = x * std.unsqueeze(dim=2) + mean.unsqueeze(dim=2)
    return out

def conv_bank(x, module_list, act, pad_type='reflect'):
    outs = []
    for layer in module_list:
        out = act(pad_layer(x, layer, pad_type))
        outs.append(out)
    out = torch.cat(outs + [x], dim=1)
    return out

def get_act(act):
    if act == 'relu':
        return nn.ReLU()
    elif act == 'lrelu':
        return nn.LeakyReLU()
    else:
        return nn.ReLU()

class MLP(nn.Module):
    def __init__(self, c_in, c_h, n_blocks, act, sn):
        super(MLP, self).__init__()
        self.act = get_act(act)
        self.n_blocks = n_blocks
        f = spectral_norm if sn else lambda x: x
        self.in_dense_layer = f(nn.Linear(c_in, c_h))
        self.first_dense_layers = nn.ModuleList([f(nn.Linear(c_h, c_h)) for _ in range(n_blocks)])
        self.second_dense_layers = nn.ModuleList([f(nn.Linear(c_h, c_h)) for _ in range(n_blocks)])

    def forward(self, x):
        h = self.in_dense_layer(x)
        for l in range(self.n_blocks):
            y = self.first_dense_layers[l](h)
            y = self.act(y)
            y = self.second_dense_layers[l](y)
            y = self.act(y)
            h = h + y
        return h

class Prenet(nn.Module):
    def __init__(self, c_in, c_h, c_out,
            kernel_size, n_conv_blocks,
            subsample, act, dropout_rate):
        super(Prenet, self).__init__()
        self.act = get_act(act)
        self.subsample = subsample
        self.n_conv_blocks = n_conv_blocks
        self.in_conv_layer = nn.Conv2d(1, c_h, kernel_size=kernel_size)
        self.first_conv_layers = nn.ModuleList([nn.Conv2d(c_h, c_h, kernel_size=kernel_size) for _ \
                in range(n_conv_blocks)])
        self.second_conv_layers = nn.ModuleList([nn.Conv2d(c_h, c_h, kernel_size=kernel_size, stride=sub)
            for sub, _ in zip(subsample, range(n_conv_blocks))])
        output_size = c_in
        for l, sub in zip(range(n_conv_blocks), self.subsample):
            output_size = ceil(output_size / sub)
        self.out_conv_layer = nn.Conv1d(c_h * output_size, c_out, kernel_size=1)
        self.dropout_layer = nn.Dropout(p=dropout_rate)
        self.norm_layer = nn.InstanceNorm2d(c_h, affine=False)

    def forward(self, x):
        # reshape x to 4D
        x = x.contiguous().view(x.size(0), 1, x.size(1), x.size(2))
        out = pad_layer_2d(x, self.in_conv_layer)
        out = self.act(out)
        out = self.norm_layer(out)
        for l in range(self.n_conv_blocks):
            y = pad_layer_2d(out, self.first_conv_layers[l])
            y = self.act(y)
            y = self.norm_layer(y)
            y = self.dropout_layer(y)
            y = pad_layer_2d(y, self.second_conv_layers[l])
            y = self.act(y)
            y = self.norm_layer(y)
            y = self.dropout_layer(y)
            if self.subsample[l] > 1:
                out = F.avg_pool2d(out, kernel_size=self.subsample[l], ceil_mode=True)
            out = y + out
        out = out.contiguous().view(out.size(0), out.size(1) * out.size(2), out.size(3))
        out = pad_layer(out, self.out_conv_layer)
        out = self.act(out)
        return out

class Postnet(nn.Module):
    def __init__(self, c_in, c_h, c_out, c_cond,
            kernel_size, n_conv_blocks,
            upsample, act, sn):
        super(Postnet, self).__init__()
        self.act = get_act(act)
        self.upsample = upsample
        self.c_h = c_h
        self.n_conv_blocks = n_conv_blocks
        f = spectral_norm if sn else lambda x: x
        total_upsample = reduce(lambda x, y: x*y, upsample)
        self.in_conv_layer = f(nn.Conv1d(c_in, c_h * c_out // total_upsample, kernel_size=1))
        self.first_conv_layers = nn.ModuleList([f(nn.Conv2d(c_h, c_h, kernel_size=kernel_size)) for _ \
                in range(n_conv_blocks)])
        self.second_conv_layers = nn.ModuleList([f(nn.Conv2d(c_h, c_h*up*up, kernel_size=kernel_size))
            for up, _ in zip(upsample, range(n_conv_blocks))])
        self.out_conv_layer = f(nn.Conv2d(c_h, 1, kernel_size=1))
        self.conv_affine_layers = nn.ModuleList(
                [f(nn.Linear(c_cond, c_h * 2)) for _ in range(n_conv_blocks*2)])
        self.norm_layer = nn.InstanceNorm2d(c_h, affine=False)
        self.ps = nn.PixelShuffle(max(upsample))

    def forward(self, x, cond):
        out = pad_layer(x, self.in_conv_layer)
        out = out.contiguous().view(out.size(0), self.c_h, out.size(1) // self.c_h, out.size(2))
        for l in range(self.n_conv_blocks):
            y = pad_layer_2d(out, self.first_conv_layers[l])
            y = self.act(y)
            y = self.norm_layer(y)
            y = append_cond_2d(y, self.conv_affine_layers[l*2](cond))
            y = pad_layer_2d(y, self.second_conv_layers[l])
            y = self.act(y)
            if self.upsample[l] > 1:
                y = self.ps(y)
                y = self.norm_layer(y)
                y = append_cond_2d(y, self.conv_affine_layers[l*2+1](cond))
                out = y + upsample(out, scale_factor=(self.upsample[l], self.upsample[l]))
            else:
                y = self.norm_layer(y)
                y = append_cond(y, self.conv_affine_layers[l*2+1](cond))
                out = y + out
        out = self.out_conv_layer(out)
        out = out.squeeze(dim=1)
        return out


In [6]:
class SpeakerEncoder1(nn.Module):
    def __init__(self, c_in, c_h, c_out, kernel_size,
            bank_size, bank_scale, c_bank,
            n_conv_blocks, n_dense_blocks,
            subsample, act, dropout_rate,
            use_hierarchy=False, n_levels=1):
        super(SpeakerEncoder1, self).__init__()
        self.c_in = c_in
        self.c_h = c_h
        self.c_out = c_out # Dimension for each individual latent level (z1, z2, etc.)
        self.kernel_size = kernel_size
        self.n_conv_blocks = n_conv_blocks
        self.n_dense_blocks = n_dense_blocks
        self.subsample = subsample
        self.act = get_act(act)
        self.use_hierarchy = use_hierarchy
        self.n_levels = n_levels

        self.conv_bank = nn.ModuleList(
                [nn.Conv1d(c_in, c_bank, kernel_size=k) for k in range(bank_scale, bank_size + 1, bank_scale)])
        in_channels = c_bank * (bank_size // bank_scale) + c_in
        self.in_conv_layer = nn.Conv1d(in_channels, c_h, kernel_size=1)

        # Convolutional blocks
        self.first_conv_layers = nn.ModuleList([nn.Conv1d(c_h, c_h, kernel_size=kernel_size) for _ in range(n_conv_blocks)])
        self.second_conv_layers = nn.ModuleList([nn.Conv1d(c_h, c_h, kernel_size=kernel_size, stride=sub) for sub in subsample])

        self.pooling_layer = nn.AdaptiveAvgPool1d(1)

        # Dense blocks
        self.first_dense_layers = nn.ModuleList([nn.Linear(c_h, c_h) for _ in range(n_dense_blocks)])
        self.second_dense_layers = nn.ModuleList([nn.Linear(c_h, c_h) for _ in range(n_dense_blocks)])

        if self.use_hierarchy:
            # For each level (z1, z2, ..., zn), we need posterior mu and log_sigma layers
            # All levels have c_out dimensions
            self.mu_layers = nn.ModuleList([nn.Linear(c_h, c_out) for _ in range(n_levels)])
            self.log_sigma_layers = nn.ModuleList([nn.Linear(c_h, c_out) for _ in range(n_levels)])

            # Prior predictor layers: `prior_predictor_layers[k]` maps from z_{k+1} to prior params for z_k
            # e.g., prior_predictor_layers[0] for z1_prior from z2.
            #       prior_predictor_layers[1] for z2_prior from z3.
            self.prior_predictor_layers = nn.ModuleList()
            for _ in range(n_levels - 1): # (n_levels - 1) prior networks needed
                self.prior_predictor_layers.append(nn.Sequential(
                    nn.Linear(c_out, c_h),
                    self.act,
                    nn.Linear(c_h, c_out * 2) # For mu_prior and log_sigma_prior
                ))
        else:
            self.output_layer = nn.Linear(c_h, c_out)

        self.dropout_layer = nn.Dropout(p=dropout_rate)

    def conv_blocks(self, inp):
        out = inp
        for l in range(self.n_conv_blocks):
            y = pad_layer(out, self.first_conv_layers[l])
            y = self.act(y)
            y = self.dropout_layer(y)
            y = pad_layer(y, self.second_conv_layers[l])
            y = self.act(y)
            y = self.dropout_layer(y)
            if self.subsample[l] > 1:
                out = F.avg_pool1d(out, kernel_size=self.subsample[l], ceil_mode=True)
            out = y + out # Residual connection
        return out

    def dense_blocks(self, inp):
        out = inp
        for l in range(self.n_dense_blocks):
            y = self.first_dense_layers[l](out)
            y = self.act(y)
            y = self.dropout_layer(y)
            y = self.second_dense_layers[l](y)
            y = self.act(y)
            y = self.dropout_layer(y)
            out = y + out # Residual connection
        return out

    def forward(self, x):
        out = conv_bank(x, self.conv_bank, act=self.act)
        out = pad_layer(out, self.in_conv_layer)
        out = self.act(out)
        out = self.conv_blocks(out)
        out = self.pooling_layer(out).squeeze(2) # Shape: (batch_size, c_h)
        pooled_features = self.dense_blocks(out) # pooled_features is (batch_size, c_h)

        if self.use_hierarchy:
            mus = []
            log_sigmas = []
            mu_priors = [] # [None (for z1), prior for z2 (from z1), prior for z3 (from z2)]
            log_sigma_priors = [] # Same
            sampled_latents = []

            # --- For z1 (Highest Level - standard normal prior) ---
            # Posterior for z1 from pooled_features
            mu_post_z1 = self.mu_layers[0](pooled_features)
            log_sigma_post_z1 = self.log_sigma_layers[0](pooled_features)

            # Sample z1
            eps_z1 = torch.randn_like(mu_post_z1)
            z1 = mu_post_z1 + torch.exp(log_sigma_post_z1 * 0.5) * eps_z1

            mus.append(mu_post_z1)
            log_sigmas.append(log_sigma_post_z1)
            sampled_latents.append(z1)

            # Prior for z1 is standard normal (N(0,I))
            mu_priors.append(None)
            log_sigma_priors.append(None)

            # --- For subsequent levels (z2, z3, etc.) ---
            # Loop from the second level (index 1) up to n_levels-1
            for i in range(1, self.n_levels):
                # Posterior for z_{i+1} from pooled_features
                mu_post_zi = self.mu_layers[i](pooled_features)
                log_sigma_post_zi = self.log_sigma_layers[i](pooled_features)

                # Sample z_{i+1}
                eps_zi = torch.randn_like(mu_post_zi)
                zi = mu_post_zi + torch.exp(log_sigma_post_zi * 0.5) * eps_zi

                mus.append(mu_post_zi)
                log_sigmas.append(log_sigma_post_zi)
                sampled_latents.append(zi) # sampled_latents[i] is z_{i+1}

                # Compute prior for z_{i+1} from z_i (sampled_latents[i-1])
                prior_params = self.prior_predictor_layers[i-1](sampled_latents[i-1])
                mu_prior_zi, log_sigma_prior_zi = prior_params.chunk(2, dim=-1)

                mu_priors.append(mu_prior_zi)
                log_sigma_priors.append(log_sigma_prior_zi)

            # The decoder input for speaker is the concatenation of all sampled speaker latents
            decoder_input_emb = torch.cat(sampled_latents, dim=-1)

            return {
                'mus': mus, # List of posterior means [mu_z1, mu_z2, mu_z3]
                'log_sigmas': log_sigmas, # List of posterior log_sigmas [log_sigma_z1, log_sigma_z2, log_sigma_z3]
                'mu_priors': mu_priors, # List of priors [None (for z1), prior_z2_from_z1, prior_z3_from_z2]
                'log_sigma_priors': log_sigma_priors, # Same
                'decoder_input': decoder_input_emb # Concatenated speaker embedding (z1 || z2 || z3 ...)
            }

        else:
            # Original non-hierarchical output remains the same
            emb = self.output_layer(pooled_features)
            return {
                'mus': [emb],
                'log_sigmas': [torch.zeros_like(emb)],
                'mu_priors': [None],
                'log_sigma_priors': [None],
                'decoder_input': emb
            }

In [7]:
class ContentEncoder1(nn.Module):
    def __init__(self, c_in, c_h, c_out, kernel_size,
            bank_size, bank_scale, c_bank,
            n_conv_blocks, subsample,
            act, dropout_rate,
            use_hierarchy=False, n_levels=1):
        super(ContentEncoder1, self).__init__()
        self.n_conv_blocks = n_conv_blocks
        self.subsample = subsample
        self.act = get_act(act)
        self.use_hierarchy = use_hierarchy
        self.n_levels = n_levels
        self.c_out = c_out # Dimension for each individual latent level (z1, z2, etc.)

        self.conv_bank = nn.ModuleList(
                [nn.Conv1d(c_in, c_bank, kernel_size=k) for k in range(bank_scale, bank_size + 1, bank_scale)])
        in_channels = c_bank * (bank_size // bank_scale) + c_in
        self.in_conv_layer = nn.Conv1d(in_channels, c_h, kernel_size=1)
        self.first_conv_layers = nn.ModuleList([nn.Conv1d(c_h, c_h, kernel_size=kernel_size) for _ in range(n_conv_blocks)])
        self.second_conv_layers = nn.ModuleList([nn.Conv1d(c_h, c_h, kernel_size=kernel_size, stride=sub) for sub in subsample])
        self.norm_layer = nn.InstanceNorm1d(c_h, affine=False)
        self.dropout_layer = nn.Dropout(p=dropout_rate)

        if self.use_hierarchy:
            # For each level (z1, z2, ..., zn), we need posterior mu and log_sigma layers
            # All levels have c_out dimensions
            self.mu_layers = nn.ModuleList([nn.Conv1d(c_h, c_out, kernel_size=1) for _ in range(n_levels)])
            self.log_sigma_layers = nn.ModuleList([nn.Conv1d(c_h, c_out, kernel_size=1) for _ in range(n_levels)])

            # Prior predictor layers: `prior_predictor_layers[k]` maps from z_{k+1} to prior params for z_k
            self.prior_predictor_layers = nn.ModuleList()
            for _ in range(n_levels - 1):
                self.prior_predictor_layers.append(nn.Sequential(
                    nn.Conv1d(c_out, c_h, kernel_size=1),
                    self.act,
                    nn.Conv1d(c_h, c_out * 2, kernel_size=1)
                ))
        else:
            self.mean_layer = nn.Conv1d(c_h, c_out, kernel_size=1)
            self.std_layer = nn.Conv1d(c_h, c_out, kernel_size=1)

    def forward(self, x):
        out = conv_bank(x, self.conv_bank, act=self.act)
        out = pad_layer(out, self.in_conv_layer) # This is correct for in_conv_layer
        out = self.norm_layer(out)
        out = self.act(out)
        out = self.dropout_layer(out)
        # convolution blocks
        for l in range(self.n_conv_blocks):
            y = pad_layer(out, self.first_conv_layers[l]) # Correct
            y = self.norm_layer(y)
            y = self.act(y)
            y = self.dropout_layer(y)
            y = pad_layer(y, self.second_conv_layers[l]) # Correct
            y = self.norm_layer(y)
            y = self.act(y)
            y = self.dropout_layer(y)
            if self.subsample[l] > 1:
                out = F.avg_pool1d(out, kernel_size=self.subsample[l], ceil_mode=True)
            out = y + out # Residual connection

        processed_features = out # (batch_size, c_h, seq_len)

        if self.use_hierarchy:
            mus = []
            log_sigmas = []
            mu_priors = [] # [None (for z1), prior for z2 (from z1), prior for z3 (from z2)]
            log_sigma_priors = [] # Same
            sampled_latents = []

            # --- For z1 (Highest Level - standard normal prior) ---
            # Posterior for z1 from processed_features
            # These are 1x1 convs, no need for external pad_layer.
            mu_post_z1 = self.mu_layers[0](processed_features)
            log_sigma_post_z1 = self.log_sigma_layers[0](processed_features)

            # Sample z1
            eps_z1 = torch.randn_like(mu_post_z1)
            z1 = mu_post_z1 + torch.exp(log_sigma_post_z1 * 0.5) * eps_z1

            mus.append(mu_post_z1)
            log_sigmas.append(log_sigma_post_z1)
            sampled_latents.append(z1)

            # Prior for z1 is standard normal (N(0,I))
            mu_priors.append(None)
            log_sigma_priors.append(None)

            # --- For subsequent levels (z2, z3, etc.) ---
            # Loop from the second level (index 1) up to n_levels-1
            for i in range(1, self.n_levels):
                # Posterior for z_{i+1} from processed_features
                # These are 1x1 convs, no need for external pad_layer.
                mu_post_zi = self.mu_layers[i](processed_features)
                log_sigma_post_zi = self.log_sigma_layers[i](processed_features)

                # Sample z_{i+1}
                eps_zi = torch.randn_like(mu_post_zi)
                zi = mu_post_zi + torch.exp(log_sigma_post_zi * 0.5) * eps_zi

                mus.append(mu_post_zi)
                log_sigmas.append(log_sigma_post_zi)
                sampled_latents.append(zi) # sampled_latents[i] is z_{i+1}

                # Compute prior for z_{i+1} from z_i (sampled_latents[i-1])
                # prior_predictor_layers[0] for z2's prior from z1
                # prior_predictor_layers[1] for z3's prior from z2
                # So, prior_predictor_layers[i-1] takes sampled_latents[i-1] (z_i)
                # FIX: Remove pad_layer here. Just call the Sequential module directly.
                prior_params = self.prior_predictor_layers[i-1](sampled_latents[i-1])
                mu_prior_zi, log_sigma_prior_zi = prior_params.chunk(2, dim=1)

                mu_priors.append(mu_prior_zi)
                log_sigma_priors.append(log_sigma_prior_zi)

            # The decoder input is always z1 (the highest level latent)
            decoder_input_content = sampled_latents[0]

            return {
                'mus': mus, # List of posterior means [mu_z1, mu_z2, mu_z3]
                'log_sigmas': log_sigmas, # List of posterior log_sigmas [log_sigma_z1, log_sigma_z2, log_sigma_z3]
                'mu_priors': mu_priors, # List of priors [None (for z1), prior_z2_from_z1, prior_z3_from_z2]
                'log_sigma_priors': log_sigma_priors, # Same
                'decoder_input': decoder_input_content # z1 only
            }

        else:
            # Original non-hierarchical output remains the same
            # These are 1x1 convs, no need for external pad_layer.
            mu = self.mean_layer(processed_features)
            log_sigma = self.std_layer(processed_features)

            eps = torch.randn_like(mu)
            z = mu + torch.exp(log_sigma * 0.5) * eps

            return {
                'mus': [mu],
                'log_sigmas': [log_sigma],
                'mu_priors': [None],
                'log_sigma_priors': [None],
                'decoder_input': z
            }

In [8]:
class Decoder(nn.Module):
    def __init__(self,
            c_in, c_cond, c_h, c_out,
            kernel_size,
            n_conv_blocks, upsample, act, sn, dropout_rate):
        super(Decoder, self).__init__()
        self.n_conv_blocks = n_conv_blocks
        self.upsample = upsample
        self.act = get_act(act)
        f = spectral_norm if sn else lambda x: x
        self.in_conv_layer = f(nn.Conv1d(c_in, c_h, kernel_size=1))
        self.first_conv_layers = nn.ModuleList([f(nn.Conv1d(c_h, c_h, kernel_size=kernel_size)) for _ \
                in range(n_conv_blocks)])
        self.second_conv_layers = nn.ModuleList(\
                [f(nn.Conv1d(c_h, c_h * up, kernel_size=kernel_size)) \
                for _, up in zip(range(n_conv_blocks), self.upsample)])
        self.norm_layer = nn.InstanceNorm1d(c_h, affine=False)
        self.conv_affine_layers = nn.ModuleList(
                [f(nn.Linear(c_cond, c_h * 2)) for _ in range(n_conv_blocks*2)])
        self.out_conv_layer = f(nn.Conv1d(c_h, c_out, kernel_size=1))
        self.dropout_layer = nn.Dropout(p=dropout_rate)

    def forward(self, z, cond):
        out = pad_layer(z, self.in_conv_layer)
        out = self.norm_layer(out)
        out = self.act(out)
        out = self.dropout_layer(out)
        # convolution blocks
        for l in range(self.n_conv_blocks):
            y = pad_layer(out, self.first_conv_layers[l])
            y = self.norm_layer(y)
            y = append_cond(y, self.conv_affine_layers[l*2](cond))
            y = self.act(y)
            y = self.dropout_layer(y)
            y = pad_layer(y, self.second_conv_layers[l])
            if self.upsample[l] > 1:
                y = pixel_shuffle_1d(y, scale_factor=self.upsample[l])
            y = self.norm_layer(y)
            y = append_cond(y, self.conv_affine_layers[l*2+1](cond))
            y = self.act(y)
            y = self.dropout_layer(y)
            if self.upsample[l] > 1:
                out = y + upsample(out, scale_factor=self.upsample[l])
            else:
                out = y + out
        out = pad_layer(out, self.out_conv_layer)
        return out


In [9]:
class AE1(nn.Module):
    def __init__(self,  args):
        super(AE1, self).__init__()
        self.use_content_hierarchy = args.use_content_hierarchy
        self.n_content_levels = args.n_content_levels if args.use_content_hierarchy else 1
        self.use_speaker_hierarchy = args.use_speaker_hierarchy
        self.n_speaker_levels = args.n_speaker_levels if args.use_speaker_hierarchy else 1

        # Configure SpeakerEncoder
        speaker_encoder_config = args.config['SpeakerEncoder'].copy()
        speaker_encoder_config['use_hierarchy'] = self.use_speaker_hierarchy
        speaker_encoder_config['n_levels'] = self.n_speaker_levels
        self.speaker_encoder = SpeakerEncoder1(**speaker_encoder_config)

        # Configure ContentEncoder
        content_encoder_config = args.config['ContentEncoder'].copy()
        content_encoder_config['use_hierarchy'] = self.use_content_hierarchy
        content_encoder_config['n_levels'] = self.n_content_levels
        self.content_encoder = ContentEncoder1(**content_encoder_config)

        # Configure Decoder
        decoder_config = args.config['Decoder'].copy()

        # Dynamically set c_in for decoder (input from ContentEncoder)
        decoder_config['c_in'] = content_encoder_config['c_out'] # z1 of content is always c_out

        # Dynamically set c_cond for decoder (input from SpeakerEncoder)
        if self.use_speaker_hierarchy:
            # Speaker decoder input is the concatenation of all speaker latents (z1 || z2 || z3 ...)
            decoder_config['c_cond'] = speaker_encoder_config['c_out'] * self.n_speaker_levels
        else:
            decoder_config['c_cond'] = speaker_encoder_config['c_out']

        self.decoder = Decoder(**decoder_config)

    def forward(self, x):
        speaker_latents_info = self.speaker_encoder(x)
        content_latents_info = self.content_encoder(x)

        # Decoder receives the specific 'decoder_input' from the encoder.
        # For speaker, it's the concatenated embedding.
        # For content, it's z1.
        speaker_decoder_input = speaker_latents_info['decoder_input']
        content_decoder_input = content_latents_info['decoder_input']

        dec = self.decoder(content_decoder_input, speaker_decoder_input)

        return {
            'dec': dec,
            'speaker_latents': speaker_latents_info,
            'content_latents': content_latents_info,
        }

    def inference(self, x, x_cond):
        speaker_latents_info = self.speaker_encoder(x_cond)
        content_latents_info = self.content_encoder(x)

        # For inference, use the posterior mean (mu) of z1 for content, as z1 is reconstructed.
        content_decoder_input = content_latents_info['mus'][0]

        # For speaker, use the `decoder_input` which is the concatenated embedding.
        speaker_decoder_input = speaker_latents_info['decoder_input']

        dec = self.decoder(content_decoder_input, speaker_decoder_input)
        return dec

    def get_speaker_embeddings(self, x):
        # This function should return the final speaker embedding used by the decoder.
        speaker_latents_info = self.speaker_encoder(x)
        return speaker_latents_info['decoder_input']

In [ ]:
#######################
###### Code    ########
#######################

In [10]:
#adaptive_voice_conversion/data_utils.py
import torch
from torch.utils.data import Dataset
import os
import pickle
import json
import numpy as np
import torch
from torch.utils.data import DataLoader

class CollateFn(object):
    def __init__(self, frame_size):
        self.frame_size = frame_size

    def make_frames(self, tensor):
        out = tensor.view(tensor.size(0), tensor.size(1) // self.frame_size, self.frame_size * tensor.size(2))
        out = out.transpose(1, 2)
        return out

    def __call__(self, l):
        data_tensor = torch.from_numpy(np.array(l))
        segment = self.make_frames(data_tensor)
        return segment

def get_data_loader(dataset, batch_size, frame_size, shuffle=True, num_workers=4, drop_last=False):
    _collate_fn = CollateFn(frame_size=frame_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
            num_workers=num_workers, collate_fn=_collate_fn, pin_memory=True)
    return dataloader

class SequenceDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.utt_ids = list(self.data.keys())

    def __getitem__(self, ind):
        utt_id = self.utt_ids[ind]
        ret = self.data[utt_id].transpose()
        return ret

    def __len__(self):
        return len(self.utt_ids)

class PickleDataset(Dataset):
    def __init__(self, pickle_path, sample_index_path, segment_size):
        with open(pickle_path, 'rb') as f:
            self.data = pickle.load(f)
        with open(sample_index_path, 'r') as f:
            self.indexes = json.load(f)
        self.segment_size = segment_size

    def __getitem__(self, ind):
        utt_id, t = self.indexes[ind]
        segment = self.data[utt_id][t:t + self.segment_size]
        return segment

    def __len__(self):
        return len(self.indexes)


In [11]:
import torch
import torch.nn.functional as F

class AELoss:
    def __init__(self, n_speaker_levels: int, n_content_levels: int):
        """
        Initializes the AELoss class.

        Args:
            n_speaker_levels: Number of speaker hierarchy levels.
            n_content_levels: Number of content hierarchy levels.
        """
        self.n_speaker_levels = n_speaker_levels
        self.n_content_levels = n_content_levels

    def calculate_kld(self, mu_post, log_sigma_post, mu_prior=None, log_sigma_prior=None):
        """
        Calculates the Kullback-Leibler Divergence (KLD).
        """
        if mu_prior is None or log_sigma_prior is None:
            # Standard normal prior (N(0, I))
            kld = -0.5 * torch.sum(1 + log_sigma_post - mu_post.pow(2) - log_sigma_post.exp())
        else:
            # Gaussian prior (N(mu_prior, exp(log_sigma_prior)))
            kld = 0.5 * torch.sum(
                torch.exp(log_sigma_post - log_sigma_prior) +
                ((mu_prior - mu_post)**2) * torch.exp(-log_sigma_prior) - 1 +
                (log_sigma_prior - log_sigma_post)
            )
        return kld

    def speaker_kld_loss(self, speaker_latents):
        """
        Calculates the KLD loss for the speaker hierarchy.
        """
        speaker_kld_loss = 0
        for i in range(self.n_speaker_levels):
            mu_post = speaker_latents['mus'][i]
            log_sigma_post = speaker_latents['log_sigmas'][i]
            mu_prior = speaker_latents['mu_priors'][i]
            log_sigma_prior = speaker_latents['log_sigma_priors'][i]
            speaker_kld_loss += self.calculate_kld(mu_post, log_sigma_post, mu_prior, log_sigma_prior)
        return speaker_kld_loss

    def content_kld_loss(self, content_latents):
        """
        Calculates the KLD loss for the content hierarchy.
        """
        content_kld_loss = 0
        for i in range(self.n_content_levels):
            mu_post = content_latents['mus'][i]
            log_sigma_post = content_latents['log_sigmas'][i]
            mu_prior = content_latents['mu_priors'][i]
            log_sigma_prior = content_latents['log_sigma_priors'][i]
            content_kld_loss += self.calculate_kld(mu_post, log_sigma_post, mu_prior, log_sigma_prior)
        return content_kld_loss

    def loss_calculate(self, x, dec, speaker_latents, content_latents, lambda_kl: float = 1.0):
        """
        Calculates the total loss for the Autoencoder.

        Args:
            x: Original input tensor.
            dec: Reconstructed output tensor from the decoder.
            speaker_latents: Dictionary containing speaker latent distributions.
            content_latents: Dictionary containing content latent distributions.
            lambda_kl: Weight for the KLD loss components.

        Returns:
            A dictionary containing individual and total loss components.
        """
        # Reconstruction Loss (e.g., L1)
        recon_loss = F.l1_loss(dec, x)

        speaker_kld_loss = self.speaker_kld_loss(speaker_latents)
        content_kld_loss = self.content_kld_loss(content_latents)

        # Total Loss with KLD weighting
        total_loss = recon_loss + lambda_kl * (speaker_kld_loss + content_kld_loss)

        return {
            'total_loss': total_loss,
            'recon_loss': recon_loss,
            'speaker_kld_loss': speaker_kld_loss,
            'content_kld_loss': content_kld_loss
        }

In [12]:
import yaml

class ARGS1():
  def __init__(self):
    # Store the YAML content directly as a string
    self.config_content = """
SpeakerEncoder:
    c_in: 80
    c_h: 128
    c_out: 128
    kernel_size: 5
    bank_size: 8
    bank_scale: 1
    c_bank: 128
    n_conv_blocks: 6
    n_dense_blocks: 6
    subsample: [1, 2, 1, 2, 1, 2]
    act: 'relu'
    dropout_rate: 0
ContentEncoder:
    c_in: 80
    c_h: 128
    c_out: 128
    kernel_size: 5
    bank_size: 8
    bank_scale: 1
    c_bank: 128
    n_conv_blocks: 6
    subsample: [1, 2, 1, 2, 1, 2]
    act: 'relu'
    dropout_rate: 0
Decoder:
    c_in: 128
    c_cond: 128
    c_h: 128
    c_out: 80
    kernel_size: 5
    n_conv_blocks: 6
    upsample: [2, 1, 2, 1, 2, 1]
    act: 'relu'
    sn: False
    dropout_rate: 0
data_loader:
    segment_size: 128
    frame_size: 1
    batch_size: 128
    shuffle: True
optimizer:
    lr: 0.0005
    beta1: 0.9
    beta2: 0.999
    amsgrad: True
    weight_decay: 0.0001
    grad_norm: 5
lambda:
    lambda_rec: 10
    lambda_kl: 1
annealing_iters: 20000
    """
    self.data_dir='/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/' # MRH: You may Modify the part
    self.train_set='train_128'
    self.train_index_file='train_samples_128.json'
    self.logdir='log/'
    self.load_model=False
    self.store_model_path='/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/checkpoint' # MRH: You may Modify the part
    self.load_model_path='/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/checkpoint' # MRH: You may Modify the part
    self.summary_steps=500
    self.save_steps=1000
    self.tag='vctk_model'
    self.iters=100000

    #  Hierarchy

    self.use_content_hierarchy=True
    self.n_content_levels=3 # MRH: in the AE1,  if use_content_hierarchy= True then n_content_levels gets number else it is 1
    self.use_speaker_hierarchy=True
    self.n_speaker_levels=3 # MRH: in the AE1,  if use_speaker_hierarchy= True then n_content_levels gets number else it is 1

    # Parse the YAML content into a dictionary
    self.config = yaml.safe_load(self.config_content)

##
args1=ARGS1()
print(args1.train_index_file)
print(args1.data_dir)

# Now you can access your config parameters like this:
print("\nAccessing config parameters:")
print(args1.config['SpeakerEncoder']['c_in'])
print(args1.config['data_loader']['batch_size'])

train_samples_128.json
/content/drive/MyDrive/AdaIN/mrh_AdaIN_Output_80bin/

Accessing config parameters:
80
128


 **Performing Speaker Verification**

**MRH: to include speaker id in output of reading data**

In [13]:
# 1. Modify PickleDataset's __getitem__ to return both utt_id and segment:

class PickleDataset(Dataset):
    def __init__(self, pickle_path, sample_index_path, segment_size):
        with open(pickle_path, 'rb') as f:
            self.data = pickle.load(f)
        with open(sample_index_path, 'r') as f:
            self.indexes = json.load(f)
        self.segment_size = segment_size

    def __getitem__(self, ind):
        utt_id, t = self.indexes[ind]
        segment = self.data[utt_id][t:t + self.segment_size]
        return segment, utt_id  ###### MRH: This is the change: Return both segment and utt_id

    def __len__(self):
        return len(self.indexes)

In [14]:
# 2. Modify CollateFn's __call__ to handle both utt_id and segment
class CollateFn(object):
    def __init__(self, frame_size):
        self.frame_size = frame_size

    def make_frames(self, tensor):
        out = tensor.view(tensor.size(0), tensor.size(1) // self.frame_size, self.frame_size * tensor.size(2))
        out = out.transpose(1, 2)
        return out

    def __call__(self, l):
        # l will be a list of (segment, utt_id) tuples from PickleDataset's __getitem__
        # Correctly extract segments (numerical data) and utt_ids (strings)
        numerical_segments = [item[0] for item in l] # This should be the numerical data
        utt_ids = [item[1] for item in l]           # This should be the string utt_id

        # Now, numerical_segments should be a list of NumPy arrays or similar,
        # which can be converted to a single NumPy array and then to a torch.Tensor.
        data_tensor = torch.from_numpy(np.array(numerical_segments))
        segment = self.make_frames(data_tensor)

        # Return the processed numerical segment tensor and the list of utt_ids
        return segment, utt_ids # MRH: Return both segment and utt_id

**Speaker Verification Class**

In [15]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import roc_curve, auc
from tqdm.auto import tqdm
import itertools # For generating pairs

class SpeakerVerificationEvaluator:
    def __init__(self, device='cpu'):
        """
        Initializes the SpeakerVerificationEvaluator.

        Args:
            device (str): The device to perform calculations on ('cuda' or 'cpu').
        """
        self.device = torch.device(device)

    def _extract_speaker_id_from_filename(self, utt_id_filename):
        """
        Extracts the speaker ID from a filename like 'p229_018.wav'.
        Assumes the speaker ID is the first part before the first underscore.
        """
        if not isinstance(utt_id_filename, str):
            raise TypeError(f"Expected utt_id_filename to be a string, but got {type(utt_id_filename)}")
        return utt_id_filename.split('_')[0]

    def _calculate_cosine_similarities(self, embeddings, speaker_ids_or_filenames):
        """
        Calculates cosine similarities for all possible pairs of embeddings.

        Args:
            embeddings (list of torch.Tensor): List of individual speaker embedding tensors.
                                               Each tensor should be 1-D, e.g., (embedding_dim,).
            speaker_ids_or_filenames (list): List of corresponding speaker IDs (strings)
                                             OR filenames (strings) from which IDs can be extracted.

        Returns:
            tuple: (similarities (np.array), labels (np.array))
                   similarities: cosine similarity scores for each pair.
                   labels: 1 for same speaker pair, 0 for different speaker pair.
        """
        if not embeddings or len(embeddings) != len(speaker_ids_or_filenames):
            raise ValueError("Embeddings and speaker_ids_or_filenames lists must be non-empty and of same length.")

        # Determine if we need to extract speaker IDs
        # We assume if the first element is a string containing '.wav', it's a filename.
        # Otherwise, assume it's already a speaker ID.
        if speaker_ids_or_filenames and isinstance(speaker_ids_or_filenames[0], str) and '.wav' in speaker_ids_or_filenames[0]:
            actual_speaker_ids = [self._extract_speaker_id_from_filename(f) for f in speaker_ids_or_filenames]
            print("Note: Speaker IDs extracted from filenames for evaluation.")
        else:
            actual_speaker_ids = speaker_ids_or_filenames
            print("Note: Using provided speaker IDs directly for evaluation.")


        n_embeddings = len(embeddings)
        print(f"Calculating similarities for {n_embeddings} embeddings. This involves "
              f"{n_embeddings * (n_embeddings - 1) // 2} pairs and can take a while for large datasets.")

        # Stack embeddings into a single tensor for potentially faster batch processing on GPU
        # Ensure embeddings are on the correct device
        embeddings_tensor = torch.stack(embeddings).to(self.device)

        similarities = []
        labels = []

        # Iterate through all unique pairs (i, j) where i < j
        # tqdm provides a progress bar
        for i, j in tqdm(itertools.combinations(range(n_embeddings), 2),
                         total=n_embeddings * (n_embeddings - 1) // 2,
                         desc="Calculating embedding pairs"):
            # Cosine similarity expects 2D tensors (batch_size, embedding_dim)
            # Unsqueeze adds a batch dimension of 1
            sim = F.cosine_similarity(embeddings_tensor[i].unsqueeze(0), embeddings_tensor[j].unsqueeze(0)).item()
            similarities.append(sim)
            labels.append(1 if actual_speaker_ids[i] == actual_speaker_ids[j] else 0)

        return np.array(similarities), np.array(labels)

    def _calculate_eer(self, fpr, tpr, thresholds):
        """
        Calculates the Equal Error Rate (EER) and the corresponding threshold.
        EER is the point where False Positive Rate (FPR) equals False Negative Rate (FNR).
        FNR = 1 - True Positive Rate (TPR).
        """
        fnr = 1 - tpr
        # Find the index where FPR and FNR are closest
        eer_threshold_idx = np.argmin(np.abs(fnr - fpr))
        eer = (fpr[eer_threshold_idx] + fnr[eer_threshold_idx]) / 2 # Average for precision
        eer_threshold = thresholds[eer_threshold_idx]
        return eer, eer_threshold

    def evaluate(self, embeddings, speaker_ids_or_filenames):
        """
        Performs speaker verification evaluation using EER and ROC AUC.

        Args:
            embeddings (list of torch.Tensor): A list of speaker embedding tensors.
                                               Each tensor should be 1-D (embedding_dim,).
                                               It's crucial to pass embeddings from a diverse set of
                                               speakers and multiple utterances per speaker for
                                               meaningful EER/ROC AUC results.
            speaker_ids_or_filenames (list): A list of corresponding speaker IDs (strings)
                                             OR filenames (strings) for each embedding.
                                             If filenames are provided (e.g., 'p229_018.wav'),
                                             the speaker ID will be extracted from them.

        Returns:
            dict: A dictionary containing:
                  'eer': Equal Error Rate.
                  'eer_threshold': The threshold at which EER occurs.
                  'roc_auc': Area Under the Receiver Operating Characteristic Curve.
                  'fpr': False Positive Rates for the ROC curve.
                  'tpr': True Positive Rates for the ROC curve.
                  'thresholds': Thresholds used for FPR/TPR calculation.
        """
        # Basic input validation
        if len(set(speaker_ids_or_filenames)) < 2: # Check unique values in the raw input list
            raise ValueError("Need embeddings from at least two different speakers for meaningful EER/ROC AUC calculation.")
        if len(embeddings) < 2:
            raise ValueError("Need at least two embeddings to form pairs for evaluation.")

        # Calculate all pairwise cosine similarities and their true labels
        similarities, labels = self._calculate_cosine_similarities(embeddings, speaker_ids_or_filenames)

        if len(set(labels)) < 2:
            # This happens if all pairs are "same speaker" or all are "different speaker"
            raise ValueError("Not enough 'same speaker' AND 'different speaker' pairs for ROC analysis. "
                             "Ensure your input data contains both types of pairs (i.e., multiple utterances "
                             "from at least one speaker, and utterances from at least two speakers).")

        # Calculate ROC curve components
        fpr, tpr, thresholds = roc_curve(labels, similarities)

        # Calculate Area Under Curve (AUC)
        roc_auc = auc(fpr, tpr)

        # Calculate EER and its threshold
        eer, eer_threshold = self._calculate_eer(fpr, tpr, thresholds)

        results = {
            'eer': eer,
            'eer_threshold': eer_threshold,
            'roc_auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
            'thresholds': thresholds
        }
        return results

**MRH: MCD metric**

In [ ]:
# !pip install "librosa==0.9.1"
!pip install pysptk

In [26]:
import numpy as np
import pysptk
from scipy.spatial.distance import euclidean
import librosa # Import librosa for DTW

class MCDMetric:
    """
    Calculates Mel-Cepstral Distortion (MCD) and Log-Spectral Distance (LSD)
    between two batches of mel-spectrograms, including DTW-based MCD.
    Lower MCD/LSD indicates better spectral similarity.
    """
    def __init__(self, n_mfcc: int = 20):
        """
        Initializes the MCDMetric.

        Args:
            n_mfcc (int): Number of Mel-Frequency Cepstral Coefficients (MFCCs)
                          to compute. Typically 12 to 24. Default is 20.
        """
        self.n_mfcc = n_mfcc
        print(f"MCDMetric initialized with n_mfcc={self.n_mfcc}")

    def _mel_to_mcc(self, mel_spec: np.ndarray) -> np.ndarray:
        """
        Converts a single mel-spectrogram to Mel-Cepstral Coefficients (MCCs).

        Args:
            mel_spec (np.ndarray): Input mel-spectrogram of shape (n_mels, n_frames).
                                   Assumed to be in dB scale (log-power).

        Returns:
            np.ndarray: MCCs of shape (n_mfcc, n_frames).
        """
        # Ensure mel_spec is 2D (n_mels, n_frames)
        # This handles cases where a single frame or a single mel-bin is passed
        if mel_spec.ndim == 1:
            mel_spec = mel_spec.reshape(-1, 1) # Treat as single frame if 1D (n_mels,)
        elif mel_spec.ndim > 2:
            # This shouldn't happen if inputs are correctly batch-indexed, but good for robustness
            raise ValueError(f"mel_spec must be 1D or 2D, but got {mel_spec.ndim}D for _mel_to_mcc.")

        # Handle empty mel_spec to prevent errors in pysptk
        if mel_spec.shape[1] == 0: # Check for n_frames == 0
            # Return an empty MCC array with correct feature dimension
            return np.empty((self.n_mfcc, 0), dtype=mel_spec.dtype)

        # Convert from dB to linear power spectrum
        # Adding a small epsilon (1e-10) to avoid issues with log(0)
        # if the mel_spec has very low (near -inf dB) or zero values.
        linear_power_spec = 10**(mel_spec / 10.0) + 1e-10

        # Transpose to (n_frames, n_mels) for pysptk, which expects rows as frames
        # and columns as spectral bins.
        # .copy(order='C') ensures it's C-contiguous, which pysptk prefers.
        # pysptk.sptk.mfcc expects a power spectrum and returns MFCCs.
        # n_mfcc-1 because the 0th coefficient (energy) is usually included,
        # so if you want 20 coefficients, the order is 19.
        mccs = pysptk.sptk.mfcc(linear_power_spec.T.copy(order='C'), order=self.n_mfcc - 1)

        # Transpose back to (n_mfcc, n_frames) for consistency
        return mccs.T

    def calculate_mcd(self, mel_spec_batch1: np.ndarray, mel_spec_batch2: np.ndarray) -> float:
        """
        Calculates the average frame-by-frame MCD between two batches of mel-spectrograms.

        Args:
            mel_spec_batch1 (np.ndarray): First batch of mel-spectrograms (e.g., ground truth/target),
                                          shape (batch_size, n_mels, n_frames).
            mel_spec_batch2 (np.ndarray): Second batch of mel-spectrograms (e.g., model output),
                                          shape (batch_size, n_mels, n_frames).

        Returns:
            float: The average Mel-Cepstral Distortion (MCD) value for the batch.
                   Returns 0.0 if no valid MCDs could be calculated or if inputs are invalid.
        """
        # --- Robustness Checks ---
        if mel_spec_batch1.shape != mel_spec_batch2.shape:
            print(f"Warning (MCD): Input mel-spectrogram batches have different shapes. "
                  f"Batch1: {mel_spec_batch1.shape}, Batch2: {mel_spec_batch2.shape}. Cannot calculate MCD.")
            return 0.0
        if mel_spec_batch1.ndim != 3 or mel_spec_batch2.ndim != 3:
             print(f"Warning (MCD): Input mel-spectrogram batches must be 3D (batch, mels, frames). "
                  f"Batch1: {mel_spec_batch1.ndim}D, Batch2: {mel_spec_batch2.ndim}D. Cannot calculate MCD.")
             return 0.0

        batch_size = mel_spec_batch1.shape[0]
        mcd_values = []

        for b in range(batch_size):
            mel_spec1 = mel_spec_batch1[b]
            mel_spec2 = mel_spec_batch2[b]

            try:
                mcc1 = self._mel_to_mcc(mel_spec1)
                mcc2 = self._mel_to_mcc(mel_spec2)
            except Exception as e:
                print(f"Error converting mel-spec to MCC for sample {b}: {e}. Skipping MCD for this sample.")
                continue

            min_frames = min(mcc1.shape[1], mcc2.shape[1])
            if min_frames == 0:
                print(f"Warning (MCD): No frames to compare for sample {b}. Skipping.")
                continue

            # Truncate to min_frames. Ensures both MCCs have same length for element-wise comparison.
            mcc1 = mcc1[:, :min_frames]
            mcc2 = mcc2[:, :min_frames]

            distances = []
            for i in range(min_frames):
                distances.append(euclidean(mcc1[:, i], mcc2[:, i]))

            if distances: # Check if list is not empty before taking mean
                mcd_values.append(np.mean(distances))

        if not mcd_values:
            # This can happen if all samples in batch failed or had 0 frames
            return 0.0

        return np.mean(mcd_values)

    def calculate_mcd_dtw(self, mel_spec_batch1: np.ndarray, mel_spec_batch2: np.ndarray) -> float:
        """
        Calculates the average DTW-based MCD between two batches of mel-spectrograms.

        Args:
            mel_spec_batch1 (np.ndarray): First batch of mel-spectrograms (e.g., ground truth/target),
                                          shape (batch_size, n_mels, n_frames).
            mel_spec_batch2 (np.ndarray): Second batch of mel-spectrograms (e.g., model output),
                                          shape (batch_size, n_mels, n_frames).

        Returns:
            float: The average DTW-MCD value for the batch.
                   Returns 0.0 if no valid MCDs could be calculated or if inputs are invalid.
        """
        # --- Robustness Checks ---
        if mel_spec_batch1.shape[0] != mel_spec_batch2.shape[0]:
            print(f"Warning (MCD-DTW): Input mel-spectrogram batches have different batch sizes. "
                  f"Batch1: {mel_spec_batch1.shape}, Batch2: {mel_spec_batch2.shape}. Cannot calculate MCD-DTW.")
            return 0.0
        if mel_spec_batch1.ndim != 3 or mel_spec_batch2.ndim != 3:
             print(f"Warning (MCD-DTW): Input mel-spectrogram batches must be 3D (batch, mels, frames). "
                  f"Batch1: {mel_spec_batch1.ndim}D, Batch2: {mel_spec_batch2.ndim}D. Cannot calculate MCD-DTW.")
             return 0.0

        batch_size = mel_spec_batch1.shape[0]
        mcd_dtw_values = []

        for b in range(batch_size):
            mel_spec1 = mel_spec_batch1[b]
            mel_spec2 = mel_spec_batch2[b]

            try:
                mcc1 = self._mel_to_mcc(mel_spec1)
                mcc2 = self._mel_to_mcc(mel_spec2)
            except Exception as e:
                print(f"Error converting mel-spec to MCC for sample {b}: {e}. Skipping MCD-DTW for this sample.")
                continue

            # Ensure there are enough frames for meaningful DTW
            # librosa.sequence.dtw will raise an error if input sequences are too short (e.g., 0 or 1 frame)
            # A common heuristic is to require at least 2 frames for DTW.
            if mcc1.shape[1] < 2 or mcc2.shape[1] < 2:
                print(f"Warning (MCD-DTW): Not enough frames for DTW for sample {b} (MCC1: {mcc1.shape[1]}, MCC2: {mcc2.shape[1]}). Skipping.")
                continue

            # Ensure MCCs are C-contiguous for librosa.dtw, it expects (features, frames)
            # .ascontiguousarray() creates a copy if not already contiguous.
            mcc1_dtw = np.ascontiguousarray(mcc1)
            mcc2_dtw = np.ascontiguousarray(mcc2)

            try:
                # D is the accumulated cost matrix, wp is the warping path
                # metric='euclidean' is the default for librosa.sequence.dtw and appropriate for MCCs.
                D, wp = librosa.sequence.dtw(X=mcc1_dtw, Y=mcc2_dtw, metric='euclidean')

                # The `wp` (warping path) can sometimes be empty if `librosa.sequence.dtw`
                # encounters very problematic inputs (e.g., extremely short, or if max_inst/max_hop
                # constraints make it impossible to find a path, though default should be fine).
                # Check if wp is not empty to avoid division by zero or errors from indexing it.
                if wp.shape[0] == 0:
                    print(f"Warning (MCD-DTW): DTW warping path is empty for sample {b}. Skipping.")
                    continue

                # Calculate the average distance along the optimal warping path.
                # Loop through the aligned frame pairs (r, c) from the warping path `wp`.
                # For each pair, calculate the Euclidean distance between the corresponding MCC vectors.
                path_distances = []
                for r, c in wp: # wp contains (row_idx_X, col_idx_Y) pairs for aligned points
                    path_distances.append(euclidean(mcc1_dtw[:, r], mcc2_dtw[:, c]))

                if path_distances: # Ensure there are distances before taking mean
                    mcd_dtw_values.append(np.mean(path_distances))
                else:
                    # This case should ideally be caught by wp.shape[0] == 0 check, but a fallback.
                    print(f"Warning (MCD-DTW): Calculated path_distances list is empty for sample {b}. Skipping.")

            except Exception as e:
                print(f"Error calculating DTW for sample {b}: {e}. Skipping.")
                continue

        if not mcd_dtw_values:
            # This can happen if all samples in batch failed or had insufficient frames
            print("Warning: No valid MCD-DTW values calculated for the entire batch. Returning 0.0.")
            return 0.0

        return np.mean(mcd_dtw_values)

    def calculate_lsd(self, mel_spec_batch1: np.ndarray, mel_spec_batch2: np.ndarray) -> float:
        """
        Calculates the average Log-Spectral Distance (LSD) between two batches of mel-spectrograms.

        Args:
            mel_spec_batch1 (np.ndarray): First batch of mel-spectrograms (e.g., ground truth/target),
                                          shape (batch_size, n_mels, n_frames).
            mel_spec_batch2 (np.ndarray): Second batch of mel-spectrograms (e.g., model output),
                                          shape (batch_size, n_mels, n_frames).

        Returns:
            float: The average Log-Spectral Distance (LSD) value for the batch.
                   Returns 0.0 if no valid LSDs could be calculated or if inputs are invalid.
        """
        # --- Robustness Checks ---
        if mel_spec_batch1.shape != mel_spec_batch2.shape:
            print(f"Warning (LSD): Input mel-spectrogram batches have different shapes. "
                  f"Batch1: {mel_spec_batch1.shape}, Batch2: {mel_spec_batch2.shape}. Cannot calculate LSD.")
            return 0.0
        if mel_spec_batch1.ndim != 3 or mel_spec_batch2.ndim != 3:
             print(f"Warning (LSD): Input mel-spectrogram batches must be 3D (batch, mels, frames). "
                  f"Batch1: {mel_spec_batch1.ndim}D, Batch2: {mel_spec_batch2.ndim}D. Cannot calculate LSD.")
             return 0.0

        batch_size = mel_spec_batch1.shape[0]
        lsd_values = []

        for b in range(batch_size):
            mel_spec1 = mel_spec_batch1[b]
            mel_spec2 = mel_spec_batch2[b]

            # Convert to (n_mels, n_frames) if needed (already handled by batch indexing)
            # Ensure inputs are treated as log-power spectra (dB scale).
            # If your mel_spec is already in dB, no explicit conversion needed here.
            # Assuming mel_spec_batch1 and mel_spec_batch2 are already in dB scale.
            log_mel_spec1 = mel_spec1
            log_mel_spec2 = mel_spec2

            # Handle empty mel_spec to prevent errors
            if log_mel_spec1.shape[1] == 0 or log_mel_spec2.shape[1] == 0:
                print(f"Warning (LSD): No frames to compare for sample {b}. Skipping.")
                continue

            try:
                # Ensure both log-mel-spectrograms have the same number of frames
                min_frames = min(log_mel_spec1.shape[1], log_mel_spec2.shape[1])

                # If there are no common frames, skip this sample
                if min_frames == 0:
                    print(f"Warning (LSD): No common frames for sample {b}. Skipping.")
                    continue

                truncated_log_mel_spec1 = log_mel_spec1[:, :min_frames]
                truncated_log_mel_spec2 = log_mel_spec2[:, :min_frames]

                # Calculate the difference between the two log-mel-spectrograms
                diff = truncated_log_mel_spec1 - truncated_log_mel_spec2

                # Square the differences
                squared_diff = np.square(diff)

                # Sum over frequency bins (axis=0 for n_mels) for each frame
                sum_squared_diff_per_frame = np.sum(squared_diff, axis=0)

                # Take the square root of the sum_squared_diff_per_frame (this is the Euclidean distance per frame)
                # Then average over frames to get the LSD for this sample
                sample_lsd = np.mean(np.sqrt(sum_squared_diff_per_frame))
                lsd_values.append(sample_lsd)

            except Exception as e:
                print(f"Error calculating LSD for sample {b}: {e}. Skipping.")
                continue

        if not lsd_values:
            print("Warning: No valid LSD values calculated for the entire batch. Returning 0.0.")
            return 0.0

        return np.mean(lsd_values)

**MRH: Some Changes to class solver to include Speaker Verification**

In [28]:
# adaptive_voice_conversion /solver.py
import torch
import numpy as np
import sys
import os
import torch.nn as nn
import torch.nn.functional as F
import yaml
import pickle
# from model import AE
# from data_utils import get_data_loader
# from data_utils import PickleDataset
# from utils import *
from functools import reduce
from collections import defaultdict
from tqdm.auto import tqdm  ############### > MRH: change

class Solver(object):
    def __init__(self, config, args):
        # print('MRH1')
        # config store the value of hyperparameters, turn to attr by AttrDict
        self.config = config
        # print(config)
        # print('MRH2')

        # args store other information
        self.args = args
        # print(self.args)
        # print(' self.args.use_content_hierarchy = ',self.args.use_content_hierarchy)
        # print('MRH3')

        # logger to use tensorboard
        self.logger = Logger(self.args.logdir)
        # print('MRH4')

        # get dataloader
        self.get_data_loaders()
        # print('MRH5')

        # init the model with config
        self.build_model()
        # print('MRH6')
        self.save_config()
        # print('MRH7')

        if args.load_model:
            self.load_model()

        # print('MRH8')

        ############### MRH: Loss
        ae1=AE1(self.args)
        self.ae_loss_calculator = AELoss(n_speaker_levels=ae1.n_speaker_levels,
                                         n_content_levels=ae1.n_content_levels)


        ####### MRH: This is the change
        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        ####
        # self.speaker_verifier = SpeakerVerificationEvaluator(device=self.args.device if hasattr(self.args, 'device') else 'cpu')
        self.speaker_verifier = SpeakerVerificationEvaluator(device)


        #################################### MRH: MCD
        # Initialize the MCD metric
        self.mcd_metric = MCDMetric(n_mfcc=20)
        # List to store MCD values for plotting
        self.mcd_values_history = [] # <-- ADD THIS LINE

    def save_model(self, iteration):
        # save model and discriminator and their optimizer
        torch.save(self.model.state_dict(), f'{self.args.store_model_path}.ckpt')
        torch.save(self.opt.state_dict(), f'{self.args.store_model_path}.opt')

    def save_config(self):
        with open(f'{self.args.store_model_path}.config.yaml', 'w') as f:
            yaml.dump(self.config, f)
        with open(f'{self.args.store_model_path}.args.yaml', 'w') as f:
            yaml.dump(vars(self.args), f)
        return

    def load_model(self):
        print(f'Load model from {self.args.load_model_path}')
        self.model.load_state_dict(torch.load(f'{self.args.load_model_path}.ckpt'))
        self.opt.load_state_dict(torch.load(f'{self.args.load_model_path}.opt'))
        return

    def get_data_loaders(self):
        data_dir = self.args.data_dir
        self.train_dataset = PickleDataset(os.path.join(data_dir, f'{self.args.train_set}.pkl'),
                os.path.join(data_dir, self.args.train_index_file),
                segment_size=self.config['data_loader']['segment_size'])
        self.train_loader = get_data_loader(self.train_dataset,
                frame_size=self.config['data_loader']['frame_size'],
                batch_size=self.config['data_loader']['batch_size'],
                shuffle=self.config['data_loader']['shuffle'],
                num_workers=4, drop_last=False)
        self.train_iter = infinite_iter(self.train_loader)


        ############################################ MRH: change

        # This is highly recommended for proper evaluation.
        # You would need to ensure your PickleDataset and get_data_loader
        # correctly yield (audio_features, speaker_id_or_filename)

        # self.val_dataset = PickleDataset(os.path.join(data_dir, f'{self.args.val_set}.pkl'),
        #                                  os.path.join(data_dir, self.args.val_index_file),
        #                                  segment_size=self.config['data_loader']['segment_size'])
        # self.val_loader = get_data_loader(self.val_dataset,
        #                                   frame_size=self.config['data_loader']['frame_size'],
        #                                   batch_size=self.config['data_loader']['batch_size'],
        #                                   shuffle=False, # No need to shuffle validation data
        #                                   num_workers=4, drop_last=False)
        # self.val_iter = infinite_iter(self.val_loader) # If you want to iterate infinitely


        return

    def build_model(self):
        # create model, discriminator, optimizers
        # print('---')
        # self.model = cc(AE1(self.config))
        self.model = cc(AE1(self.args))
        # print('***')
        # print(self.model)
        optimizer = self.config['optimizer']
        self.opt = torch.optim.Adam(self.model.parameters(),
                lr=optimizer['lr'], betas=(optimizer['beta1'], optimizer['beta2']),
                amsgrad=optimizer['amsgrad'], weight_decay=optimizer['weight_decay'])
        # print(self.opt)
        return
    ########################################################
    # Assuming 'model' is an instance of AE and 'batch' contains input 'x'
    # This should be part of your training loop
    # def ae_step(model, x):
    def ae_step(self,x,lambda_kl):
        x = cc(x)
        output = self.model(x)
        dec = output['dec']
        speaker_latents = output['speaker_latents']
        content_latents = output['content_latents']

        # # Reconstruction Loss (e.g., L1, MSE)
        # recon_loss = F.l1_loss(dec, x) # Or F.mse_loss(dec, x)

        # # KLD Loss for Speaker Hierarchy
        # speaker_kld_loss = 0
        # # Loop for each level (0 for z1, 1 for z2, etc.)
        # for i in range(self.model.n_speaker_levels):
        #     mu_post = speaker_latents['mus'][i]
        #     log_sigma_post = speaker_latents['log_sigmas'][i]

        #     # For z1 (i=0), mu_prior and log_sigma_prior will be None (standard normal)
        #     # For z2 (i=1), mu_prior and log_sigma_prior will be conditioned on z1
        #     # For z3 (i=2), mu_prior and log_sigma_prior will be conditioned on z2
        #     mu_prior = speaker_latents['mu_priors'][i]
        #     log_sigma_prior = speaker_latents['log_sigma_priors'][i]

        #     speaker_kld_loss += calculate_kld(mu_post, log_sigma_post, mu_prior, log_sigma_prior)

        # # KLD Loss for Content Hierarchy
        # content_kld_loss = 0
        # # Loop for each level (0 for z1, 1 for z2, etc.)
        # for i in range(self.model.n_content_levels):
        #     mu_post = content_latents['mus'][i]
        #     log_sigma_post = content_latents['log_sigmas'][i]

        #     # For z1 (i=0), mu_prior and log_sigma_prior will be None (standard normal)
        #     # For z2 (i=1), mu_prior and log_sigma_prior will be conditioned on z1
        #     # For z3 (i=2), mu_prior and log_sigma_prior will be conditioned on z2
        #     mu_prior = content_latents['mu_priors'][i]
        #     log_sigma_prior = content_latents['log_sigma_priors'][i]

        #     content_kld_loss += calculate_kld(mu_post, log_sigma_post, mu_prior, log_sigma_prior)

        # # Total Loss
        # # You might want to weigh KLD losses
        # total_loss = recon_loss + speaker_kld_loss + content_kld_loss

        # return total_loss, recon_loss, speaker_kld_loss, content_kld_loss

        loss_components = self.ae_loss_calculator.loss_calculate(x, dec, speaker_latents, content_latents, lambda_kl)
        loss= loss_components['total_loss']

        ###
        self.opt.zero_grad()
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(self.model.parameters(),
                max_norm=self.config['optimizer']['grad_norm'])
        self.opt.step()

        ###

        return loss_components, output

        # return {
        #     'recon_loss': recon_loss,
        #     'speaker_kld_loss': speaker_kld_loss,
        #     'content_kld_loss': content_kld_loss
        #     # You might add other relevant values here if they are directly calculated
        #     # and consistently available within ae_step (e.g., total_loss and grad_norm
        #     # if ae_step also handled optimization, but we decided against that earlier).
        # }
################################################################################
    def train(self, n_iterations):
        # print('mrh 1')
        batch_mcd = 0.0 # Initialize with a default value
        batch_mcd_dtw = 0.0
        batch_lsd=0.0
        for iteration in range(n_iterations):
            # print('mrh 2')
            # print('iteration= ',iteration)
            if iteration >= self.config['annealing_iters']:
                lambda_kl = self.config['lambda']['lambda_kl']
            else:
                lambda_kl = self.config['lambda']['lambda_kl'] * (iteration + 1) / self.config['annealing_iters']
            data = next(self.train_iter)
            # print('mrh 3')
            # print('data= ',data)
            # print('\ndata length= ',len(data))
            # print('\ndata type= ',type(data))
            # print('\ndata[0].shape=  ', data[0].shape)
            # print('\nlen(data[1])=  ', len(data[1]))

            # raise Exception('Mohammad Reza Hasanabadi')

            ######################################################### MRH: change
            # meta = self.ae_step(data, lambda_kl)
            meta , output= self.ae_step(data[0] if isinstance(data, (list, tuple)) else data, lambda_kl)


            # print('meta type = ',type(meta))
            # print('mrh 4')
            # add to logger
            if iteration % self.args.summary_steps == 0:
                # print('mrh 5')
                self.logger.scalars_summary(f'{self.args.tag}/ae_train', meta, iteration)
                # print('mrh 6')
            # loss_rec = meta['loss_rec']
            loss_rec = meta['recon_loss']
            # print('mrh 7')
            # loss_kl = meta['loss_kl']
            speaker_loss_kl = meta['speaker_kld_loss']
            content_loss_kl = meta['content_kld_loss']
            # print('mrh 8')

            # print(f'AE:[{iteration + 1}/{n_iterations}], loss_rec={loss_rec:.2f}, '
                    # f'loss_kl={loss_kl:.2f}, lambda={lambda_kl:.1e}     ', end='\r')


            if (iteration + 1) % self.args.save_steps == 0 or iteration + 1 == n_iterations:
                self.save_model(iteration=iteration)
                print()

            ############################################################################# MRH: change

            if (iteration + 1) % 10 == 0 :
                self.model.eval() # Still good practice to put model in eval mode for this

                # print('1')
                audio_data_batch = data[0] # Use the current batch's audio data
                speaker_ids_batch = data[1]                 # Use the current batch's speaker IDs

                all_embeddings = []
                all_speaker_ids = []

                # print('2')
                # with torch.no_grad():
                audio_data_batch = cc(audio_data_batch)
                # print('audio_data_batch shape =',audio_data_batch.shape)
                model_output = self.model(audio_data_batch)
                # print('22')
                embeddings_batch = model_output['speaker_latents']['decoder_input']

                # Collect individual embeddings and speaker IDs from the current batch
                print('3')
                for i in range(embeddings_batch.size(0)):
                    all_embeddings.append(embeddings_batch[i].cpu())
                    all_speaker_ids.append(speaker_ids_batch[i])

                print('4')
                try:
                    # Check if there are enough distinct speakers for a meaningful EER
                    if len(set(all_speaker_ids)) < 2 or len(all_embeddings) < 2:
                        print("Skipping batch-level speaker verification: Not enough unique speakers or samples in batch.")
                    else:
                        eval_results = self.speaker_verifier.evaluate(all_embeddings, all_speaker_ids)
                        # Log with a different tag to distinguish from full eval
                        self.logger.scalars_summary(f'{self.args.tag}/speaker_verification_batch', {
                            'eer': eval_results['eer'],
                            'roc_auc': eval_results['roc_auc'],
                            'eer_threshold': eval_results['eer_threshold']
                        }, iteration)
                        print(f"Batch Speaker Verification Results: EER={eval_results['eer']:.4f}, ROC AUC={eval_results['roc_auc']:.4f}")
                except ValueError as e:
                    print(f"Skipping batch-level speaker verification due to data inadequacy: {e}")

                self.model.train()
                print("-----------------------------------------------------------------------------------\n")
            # --- End Speaker Verification on Current Batch ---



            ###################################################### MRH: MCD
                        # Add to logger and calculate MCD only when summary_steps condition is met
            if iteration % 5 == 0:
                # self.logger.scalars_summary(f'{self.args.tag}/ae_train', meta, iteration)

                # --- Start: MCDMetric Calculation Moved Inside Summary Block ---

                # Convert PyTorch tensors to NumPy arrays for metric calculation
                # Ensure they are on CPU and detached from graph
                source_mels = data[0].cpu().detach().numpy()
                converted_mels = output['dec'].cpu().detach().numpy()

                # Calculate MCD for the current batch using the initialized mcd_metric instance
                batch_mcd = self.mcd_metric.calculate_mcd(source_mels, converted_mels)


                # Calculate MCD-DTW
                batch_mcd_dtw = self.mcd_metric.calculate_mcd_dtw(source_mels, converted_mels)

                # Calculate lsd
                batch_lsd = self.mcd_metric.calculate_lsd(source_mels, converted_mels)

                # Log MCD
                # self.logger.scalars_summary(f'{self.args.tag}/metrics/mcd', {'mcd': batch_mcd}, iteration)

                # Store the calculated MCD value
                self.mcd_values_history.append(batch_mcd) # <-- STORE HERE

                # Print the average MCD immediately
                # print(f"MCD calculated at iteration {iteration}: {batch_mcd:.4f}") # <-- PRINT HERE

                # --- End: MCDMetric Calculation Moved Inside Summary Block ---


            ####################################

            print(f'AE:[{iteration + 1}/{n_iterations}], loss_rec={meta["recon_loss"]:.2f}, '
                    f'speaker_loss_kl={meta["speaker_kld_loss"]:.2f}, content_loss_kl={meta["content_kld_loss"]:.2f} ,'
                    f'lambda={lambda_kl:.1e}, MCD={batch_mcd:.4f}, MCD_DTW={batch_mcd_dtw:.4f}, batch_lsd={batch_lsd:.4f}') # This will print 0.0 if not calculated in this iter.

        return

In [ ]:
from argparse import ArgumentParser, Namespace
import torch
# from solver import Solver # Make sure Solver is defined or imported
import yaml
import sys

# Assume your ARGS class (with the embedded config) is defined above or imported

if __name__ == '__main__':
    args1 = ARGS1() # Instantiate your ARGS class

    # The config is already loaded into args.config within the ARGS __init__
    # So, you just need to assign it to a local variable 'config' if you want
    config = args1.config

    # Now, pass this 'config' dictionary to your Solver
    solver = Solver(config=config, args=args1)

    if args1.iters > 0:
        solver.train(n_iterations=args1.iters)